# Net fusion results analysis
The notebook analyzes the results of fusing different models results in different combinations

## Libraries loading

In [1]:
import os
import ntpath
import sys
if '../' not in sys.path:
    sys.path.append('../')
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
import sklearn.metrics as M
from sklearn.metrics import log_loss
from scipy.special import expit
import numpy as np
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, train_test_split
from itertools import combinations
from isplutils.utils import aggregate
from multiprocessing import Pool, cpu_count
import seaborn as sns
from p_tqdm import p_map

## Parameters

In [2]:
results_root = Path('/nas/public/exchange/icpr2020/results/')
results_model_folder = list(results_root.glob('net-*'))
column_list = ['video', 'score', 'label']
do_distplot = False

## Helper functions

In [3]:
def compute_metrics(df_res:pd.DataFrame,train_tag:str) -> dict:
    numreal = sum(df_res['label']==False)
    numfake = sum(df_res['label']==True
)
    
    netname = train_tag.split('net-')[1].split('_')[0]
    traindb = train_tag.split('traindb-')[1].split('_')[0]
    
    loss = M.log_loss(df_res['label'],expit(df_res['score']))
    acc = M.accuracy_score(df_res['label'],df_res['score']>0)
    accbal = M.balanced_accuracy_score(df_res['label'],df_res['score']>0)
    rocauc = M.roc_auc_score(df_res['label'],df_res['score'])
    
    res_dict = {'traintag':train_tag,
                'net':netname,
                'traindb': traindb,
                'testdb':testdb,'testsplit':testsplit,
                'numreal':numreal,'numfake':numfake,
                'loss':loss,
                'acc':acc,'accbal':accbal,
                'rocauc':rocauc} 
    return res_dict

def highlight_values(s):
    '''
    highlight the three lowest and highest values in a Series (min: [red, yellow, darkorange], max: [green, lime, aqua]).
    '''
    s_idx = np.argsort(s.values)
    style = [''] * len(s)
    style[s_idx[0]] = 'background-color: red'
#     style[s_idx[1]] = 'background-color: yellow'
#     style[s_idx[2]] = 'background-color: darkorange'
    style[s_idx[-1]] = 'background-color: green'
#     style[s_idx[-2]] = 'background-color: lime'
#     style[s_idx[-3]] = 'background-color: aqua'
    return style

## Load data

In [4]:
# Load data in multi-index dataframe
if os.path.exists('data_frame_df.pkl'):
    data_frame_df = pd.read_pickle('data_frame_df.pkl')
    model_list = []
    for model_folder in tqdm(results_model_folder):
        dataset_list = []
        train_model_tag = model_folder.name
        model_results = model_folder.glob('*.pkl')
        for model_path in model_results:
            dataset_tag = os.path.splitext(ntpath.split(model_path)[1])[0]
            dataset_list.append(dataset_tag)
        model_list.append(train_model_tag)
else:
    data_model_list = []
    model_list = []
    for model_folder in tqdm(results_model_folder):
        data_dataset_list = []
        dataset_list = []
        train_model_tag = model_folder.name
        model_results = model_folder.glob('*.pkl')
        for model_path in model_results:
            netname = train_model_tag.split('net-')[1].split('_')[0]
            traindb = train_model_tag.split('traindb-')[1].split('_')[0]
            testdb, testsplit = model_path.with_suffix('').name.rsplit('_',1)
            dataset_tag = os.path.splitext(ntpath.split(model_path)[1])[0]
            df_frames = pd.read_pickle(model_path)[column_list]
            # Add info on training and test datasets
            df_frames['netname'] = netname
            df_frames['train_db'] = traindb
            df_frames['test_db'] = testdb
            df_frames['test_split'] = testsplit
            data_dataset_list.append(df_frames)
            dataset_list.append(dataset_tag)
        data_model_list.append(pd.concat(data_dataset_list, keys=dataset_list, names=['dataset']))
        model_list.append(train_model_tag)
    data_frame_df = pd.concat(data_model_list, keys=model_list, names=['model']).swaplevel(0, 1)
    data_frame_df.to_pickle('data_frame_df.pkl')

### Eliminate cross-datasets experiments (we already know they don't work) and Xception

In [5]:
# data_frame_df = data_frame_df[data_frame_df['train_db']==data_frame_df['test_db']]
data_frame_df = data_frame_df[data_frame_df['test_split']=='test']
data_frame_df = data_frame_df[data_frame_df['netname'] != 'Xception']
model_list = [x for x in model_list if "Xception_" not in x]
dataset_list = [x for x in dataset_list if "_val" not in x]
data_frame_df

video  \
dataset                 model                                              facepath                                                             
ff-c23-720-140-140_test net-EfficientNetB4_traindb-ff-c23-720-140-140_f... manipulated_sequences/Deepfakes/c23/videos/134_...            9206   
                                                                           manipulated_sequences/Deepfakes/c23/videos/134_...            9206   
                                                                           manipulated_sequences/Deepfakes/c23/videos/134_...            9206   
                                                                           manipulated_sequences/Deepfakes/c23/videos/134_...            9206   
                                                                           manipulated_sequences/Deepfakes/c23/videos/134_...            9206   
...                                                                                                                                       ...   
dfdc-35-5-10_test       net-EfficientNetAutoAttB4ST_traindb-dfdc-35-5-1... dfdc_train_part_47/zpujdfwivh.mp4/fr260_subj2.jpg   zpujdfwivh.mp4   
                                                                           dfdc_train_part_47/zpujdfwivh.mp4/fr270_subj2.jpg   zpujdfwivh.mp4   
                                                                           dfdc_train_part_47/zpujdfwivh.mp4/fr279_subj2.jpg   zpujdfwivh.mp4   
                                                                           dfdc_train_part_47/zpujdfwivh.mp4/fr289_subj2.jpg   zpujdfwivh.mp4   
                                                                           dfdc_train_part_47/zpujdfwivh.mp4/fr299_subj2.jpg   zpujdfwivh.mp4   

                                                                                                                                  score  \
dataset                 model                                              facepath                                                       
ff-c23-720-140-140_test net-EfficientNetB4_traindb-ff-c23-720-140-140_f... manipulated_sequences/Deepfakes/c23/videos/134_...  3.556629   
                                                                           manipulated_sequences/Deepfakes/c23/videos/134_...  1.067607   
                                                                           manipulated_sequences/Deepfakes/c23/videos/134_...  3.871128   
                                                                           manipulated_sequences/Deepfakes/c23/videos/134_...  3.076689   
                                                                           manipulated_sequences/Deepfakes/c23/videos/134_...  4.006863   
...                                                                                                                                 ...   
dfdc-35-5-10_test       net-EfficientNetAutoAttB4ST_traindb-dfdc-35-5-1... dfdc_train_part_47/zpujdfwivh.mp4/fr260_subj2.jpg   3.383530   
                                                                           dfdc_train_part_47/zpujdfwivh.mp4/fr270_subj2.jpg   4.214155   
                                                                           dfdc_train_part_47/zpujdfwivh.mp4/fr279_subj2.jpg   3.685401   
                                                                           dfdc_train_part_47/zpujdfwivh.mp4/fr289_subj2.jpg  -0.509255   
                                                                           dfdc_train_part_47/zpujdfwivh.mp4/fr299_subj2.jpg   3.152403   

                                                                                                                               label  \
dataset                 model                                              facepath                                                    
ff-c23-720-140-140_test net-EfficientNetB4_traindb-ff-c23-720-140-140_f... manipulated_sequences/Deepfakes/c23/videos/134_...   True   
                                  

## Two models

In [6]:
print(dataset_list)
# index_list = ['bce_'+dataset_list[0], 'bce_'+dataset_list[1], 'acc_'+dataset_list[0], 'acc_'+dataset_list[1], 'accbal_'+dataset_list[0], 
#               'accbal_'+dataset_list[1], 'rocauc_'+dataset_list[0], 'rocauc_'+dataset_list[1]] # complete index list
index_list = ['bce_'+dataset_list[0], 'bce_'+dataset_list[1], 'rocauc_'+dataset_list[0], 'rocauc_'+dataset_list[1]] # partial index list without accuracies (not important for now)
print(index_list)

['ff-c23-720-140-140_test', 'dfdc-35-5-10_test']
['bce_ff-c23-720-140-140_test', 'bce_dfdc-35-5-10_test', 'rocauc_ff-c23-720-140-140_test', 'rocauc_dfdc-35-5-10_test']


#### Per video loss

In [24]:
results_two_df_video = pd.DataFrame(index=index_list)

for comb in tqdm(list(combinations(np.arange(len(model_list)), 2))):
    model_comb = np.asarray(model_list)[list(comb)]
    method = 'two_{:s}_sm'.format(str(comb)[1:-1].replace(', ', '-'))
    results_two_df[method] = np.nan
    if do_distplot:
        print('--------------------------------------------------------------------------------------------------------')
        print(model_comb)
        plt.figure(figsize=(16, 3))
    
    loss = []
#     acc = []
#     accbal = []
    rocauc = []
    for d_idx, dataset in enumerate(dataset_list):
        # Aggregate
        group_df = data_frame_df.loc[dataset].loc[model_comb].groupby('video')
        y_pred = group_df['score'].apply(lambda x: expit(np.mean(np.array(x))))
        y_true = group_df['label'].apply(lambda x: np.array(x)[0])
        
        # Compute loss
        loss.append(log_loss(y_true, y_pred))
#         acc.append(M.accuracy_score(y_true, y_pred>0))
#         accbal.append(M.balanced_accuracy_score(y_true, y_pred>0))
        rocauc.append(M.roc_auc_score(y_true, y_pred))
        rocaucexpit.append(M.roc_auc_score(y_true, expit(y_pred)))
        
        # Plot
        if do_distplot:
            plt.subplot(1, 3, d_idx+1)
            sns.distplot(y_pred[y_true == True], kde_kws={'clip': (0, 1)})
            sns.distplot(y_pred[y_true == False], kde_kws={'clip': (0, 1)})
            plt.title('{:s}: {:.4f}'.format(dataset, loss))
    
    
#     results_two_df.at[index_list, method] = [loss[0], loss[1], acc[0], acc[1], accbal[0], accbal[1], rocauc[0], rocauc[1]]
    results_two_df_videos.at[index_list, method] = [loss[0], loss[1], rocauc[0], rocauc[1]]
    
    if do_distplot:
        plt.show()

#### Per frames loss

In [15]:
from p_tqdm import p_map
results_two_df_frames = pd.DataFrame(index=index_list)

def applyParallel(dfGrouped, func):
    with Pool(12) as p:
        ret_list = p.map(func, [group for name, group in dfGrouped])
    return np.concatenate(ret_list)

def compute_mean_score(x):
    return expit(np.mean(x.to_numpy()))

def compute_label(x):
    return x.to_numpy()[0]

# def compute_partial_results(comb):
#     model_comb = np.asarray(model_list)[list(comb)]
#     method = 'two_{:s}_sm'.format(str(comb)[1:-1].replace(', ', '-'))
#     partial_res = pd.DataFrame(index=index_list)
#     if do_distplot:
#         print('--------------------------------------------------------------------------------------------------------')
#         print(model_comb)
#         plt.figure(figsize=(16, 3))
    
#     loss = []
#     rocauc = []
#     for d_idx, dataset in enumerate(dataset_list):
#         # Aggregate
#         group_df = data_frame_df.loc[dataset].loc[model_comb].groupby('facepath')
#         y_pred = [expit(np.mean(group.to_numpy())) for _, group in group_df['score']]
#         y_true = [group.to_numpy()[0] for _, group in group_df['label']]
        
#         # Compute loss
#         loss.append(log_loss(y_true, y_pred))
#         rocauc.append(M.roc_auc_score(y_true, y_pred))
        
#         # Plot
#         if do_distplot:
#             plt.subplot(1, 3, d_idx+1)
#             sns.distplot(y_pred[y_true == True], kde_kws={'clip': (0, 1)})
#             sns.distplot(y_pred[y_true == False], kde_kws={'clip': (0, 1)})
#             plt.title('{:s}: {:.4f}'.format(dataset, loss))
        
#     partial_res.at[index_list, method] = [loss[0], loss[1], rocauc[0], rocauc[1]]
#     return partial_res

# # with Pool(12) as p:
# #     partial_results = p.map(compute_partial_results, list(combinations(np.arange(len(model_list)), 2)))

# partial_results = p_map(compute_partial_results, list(combinations(np.arange(len(model_list)), 2)), num_cpus=12)

# results_two_df_frames = pd.concat(partial_results)
    
for comb in tqdm(list(combinations(np.arange(len(model_list)), 2))):
    model_comb = np.asarray(model_list)[list(comb)]
    method = 'two_{:s}_sm'.format(str(comb)[1:-1].replace(', ', '-'))
    results_two_df_frames[method] = np.nan
    if do_distplot:
        print('--------------------------------------------------------------------------------------------------------')
        print(model_comb)
        plt.figure(figsize=(16, 3))
    
    loss = []
    rocauc = []
    for d_idx, dataset in enumerate(dataset_list):
        # Aggregate
        group_df = data_frame_df.loc[dataset].loc[model_comb].groupby('facepath')
        y_pred = applyParallel(group_df['label'], compute_mean_score)
        y_true = applyParallel(group_df['label'], compute_label)
        
        # Compute loss
        loss.append(log_loss(y_true, y_pred))
        rocauc.append(M.roc_auc_score(y_true, y_pred))
        
        # Plot
        if do_distplot:
            plt.subplot(1, 3, d_idx+1)
            sns.distplot(y_pred[y_true == True], kde_kws={'clip': (0, 1)})
            sns.distplot(y_pred[y_true == False], kde_kws={'clip': (0, 1)})
            plt.title('{:s}: {:.4f}'.format(dataset, loss))
    
    
    results_two_df_frames.at[index_list, method] = [loss[0], loss[1], rocauc[0], rocauc[1]]
    
    if do_distplot:
        plt.show()

Process ForkPoolWorker-3:
Process ForkPoolWorker-8:
Process ForkPoolWorker-6:
Process ForkPoolWorker-4:
Process ForkPoolWorker-7:
Process ForkPoolWorker-5:
Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-9:
Process ForkPoolWorker-12:
Process ForkPoolWorker-11:
Process ForkPoolWorker-10:
  File "/nas/home/ecannas/miniconda3/envs/dfdc/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/nas/home/ecannas/miniconda3/envs/dfdc/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/nas/home/ecannas/miniconda3/envs/dfdc/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/nas/home/ecannas/miniconda3/envs/dfdc/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/nas/home/ecannas/miniconda3/envs/dfdc/lib/py

  File "/nas/home/ecannas/miniconda3/envs/dfdc/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
KeyboardInterrupt
KeyboardInterrupt
  File "/nas/home/ecannas/miniconda3/envs/dfdc/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/nas/home/ecannas/miniconda3/envs/dfdc/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/nas/home/ecannas/miniconda3/envs/dfdc/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/nas/home/ecannas/miniconda3/envs/dfdc/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/nas/home/ecannas/miniconda3/envs/dfdc/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/nas/home/ecannas/miniconda3/envs/dfdc/lib/python3.6/multiprocessing

KeyboardInterrupt: 

In [10]:
results_two_df_frames

,two_0-1_sm,two_0-2_sm,two_0-3_sm,two_0-4_sm,two_0-5_sm,two_0-6_sm,two_0-7_sm,two_0-8_sm,two_0-9_sm,two_0-10_sm,...,two_7-8_sm,two_7-9_sm,two_7-10_sm,two_7-11_sm,two_8-9_sm,two_8-10_sm,two_8-11_sm,two_9-10_sm,two_9-11_sm,two_10-11_sm
bce_ff-c23-720-140-140_test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bce_dfdc-35-5-10_test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rocauc_ff-c23-720-140-140_test,0.919543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rocauc_dfdc-35-5-10_test,0.827018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bce_ff-c23-720-140-140_test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rocauc_dfdc-35-5-10_test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.862528,NaN
bce_ff-c23-720-140-140_test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bce_dfdc-35-5-10_test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rocauc_ff-c23-720-140-140_test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.658942


In [11]:
comb = list(combinations(np.arange(len(model_list)), 2))
model_comb = np.asarray(model_list)[list(comb[0])]
group_df = data_frame_df.loc[dataset_list[1]].loc[model_comb].groupby('facepath')

In [12]:
iterator = group_df['score'].__iter__()
groups = [expit(np.mean(group.to_numpy())) for _, group in group_df['score']]
groups_label = [group.to_numpy()[0] for _, group in group_df['label']]
# print(name, group)
# print(groups)

In [14]:
print(type(groups[0]))
print(groups[0], len(groups))
print(type(groups_label[0]))
print(groups_label[0], len(groups_label))
print(log_loss(groups_label, groups))
print(M.roc_auc_score(groups_label, groups))

<class 'numpy.float32'>
0.60053957 949084
<class 'numpy.bool_'>
True 949084
0.7129541360267244
0.8270184246448854


In [25]:
results_two_df_frames.T.style.apply(highlight_values)

,bce_ff-c23-720-140-140_test,bce_dfdc-35-5-10_test,rocauc_ff-c23-720-140-140_test,rocauc_dfdc-35-5-10_test,rocaucexpit_ff-c23-720-140-140_test,rocaucexpit_dfdc-35-5-10_test
two_0-1_sm,0.336162,0.569402,0.951224,0.904609,0.951224,0.904609
two_0-2_sm,0.231045,1.210614,0.969713,0.723877,0.969713,0.723877
two_0-3_sm,0.353878,0.439426,0.942449,0.929469,0.942449,0.929469
two_0-4_sm,0.245556,1.170922,0.967895,0.722120,0.967895,0.722120
two_0-5_sm,0.377685,0.433455,0.942551,0.934999,0.942551,0.934999
two_0-6_sm,0.217130,1.192079,0.972392,0.745298,0.972392,0.745298
two_0-7_sm,0.232154,1.158846,0.968342,0.726571,0.968342,0.726571
two_0-8_sm,0.236542,1.191757,0.968036,0.728919,0.968036,0.728919
two_0-9_sm,0.385053,0.463961,0.947105,0.923808,0.947105,0.923808
two_0-10_sm,0.331329,0.688243,0.957353,0.857704,0.957353,0.857704


### Weight two models

In [17]:
def get_best_loss_weights(y_true, y_pred):
    loss_list = []
    a_list = np.linspace(0, 1, 30)
    for a in a_list:
        loss_list.append( log_loss(y_true, expit(y_pred.T.dot([a, 1-a])) ) )
    best_idx = np.argmin(loss_list)
    return loss_list[best_idx], a_list[best_idx]

def get_best_rocauc_weights(y_true, y_pred):
    auc_list = []
    a_list = np.linspace(0, 1, 30)
    for a in a_list:
        loss_list.append(M.roc_auc_score(y_true, y_pred.T.dot([a, 1-a])))
    best_idx = np.argmin(loss_list)
    return loss_list[best_idx], a_list[best_idx]

def get_best_rocaucexpit_weights(y_true, y_pred):
    auc_list = []
    a_list = np.linspace(0, 1, 30)
    for a in a_list:
        loss_list.append(M.roc_auc_score(y_true, expit(y_pred.T.dot([a, 1-a]))))
    best_idx = np.argmin(loss_list)
    return loss_list[best_idx], a_list[best_idx]

In [10]:
# Aggregate
loss_list = []
comb_list = []
model_comb_list = []
a_list = []
for comb in tqdm(list(combinations(np.arange(len(model_list)), 2))):
    model_comb = np.asarray(model_list)[list(comb)]
    y_pred = []
    y_true = []
    for model in model_comb:
        group_df = data_frame_df.loc['dfdc_val'].loc[model].groupby('video')
        y_pred.append(group_df['score'].apply(lambda x: np.array(x).mean()))
        y_true.append(group_df['label'].apply(lambda x: np.array(x)[0]))
    y_true = np.array(y_true).mean(axis=0)
    y_pred = np.array(y_pred)
    loss, a = get_best_weights(y_true, y_pred)
    
    
    loss_list.append(loss)
    a_list.append(a)
    comb_list.append(comb)
    model_comb_list.append(model_comb)

KeyError: 'dfdc_val'

In [ ]:
#print(loss_list)
#print(a_list)
#print(comb_list)
#print()

idx = np.argmin(loss_list)
print(model_comb_list[idx])
print(loss_list[idx], a_list[idx])